motive: We are attempting to use a constant-scaling method to perform endpoint estimations for beta and (possibly) gamma distributions. 
Basic steps are as follows:
1. Simulate a bunch of points and take some upper proportion p (upper thirds, upper quartile, etc.)
2. Calculate the standard deviation of the sample
3. Figure out what constant $c * \hat{\sigma}$ added to the sample maximum would get close to 1
4. Repeat 1-3 and obtain average of $c$ s.
5. Test on different Beta distributions (and maybe Gamma distributions). Is there a general rule?

In [10]:
#Import libraries here
import numpy as np 
import scipy.stats as sc

In [11]:
#Start with one instance of Beta(2,5) distribution
rng = np.random.default_rng(seed=2024)
sample = rng.beta(2,5,1000) #sample 1000 points from distribution
cutoff = np.percentile(sample, 66.7) #find cutoff for top third
test_sample = sample[sample > cutoff] #get top third
sd = np.std(test_sample) #get standard deviation
c = (1-np.max(test_sample)) / sd #get constant
print(c)

1.4094303685479377


In [12]:
#Use different seeds (obtain different samples) to get average value of c
cs = []
for seed in [1356,1023,1748,489,1265,31,1776,888,1901,1437,619,1975,509,1342,745,1623,1288,1755,94,1389]:
    rng = np.random.default_rng(seed)
    sample = rng.beta(2,5,1000)
    cutoff = np.percentile(sample, 66.7) 
    test_sample = sample[sample > cutoff]
    sd = np.std(test_sample)
    c = (1-np.max(test_sample)) / sd
    cs.append(c)
print(np.average(cs)) #1.5?

1.4923027364652388


In [13]:
#Test for variation in different sample sizes
cs = []
for size in [200,500,1000,5000,10000,20000,50000,int(1e5),int(1e6)]:
    for seed in [1356,1023,1748,489,1265,31,1776,888,1901,1437,619,1975,509,1342,745,1623,1288,1755,94,1389]:
        rng = np.random.default_rng(seed)
        sample = rng.beta(2,5,size)
        cutoff = np.percentile(sample, 66.7) 
        test_sample = sample[sample > cutoff]
        sd = np.std(test_sample)
        c = (1-np.max(test_sample)) / sd
        cs.append(c)
    print(np.average(cs)) #c changes for some reason, but overall decreasing trend (tends to 1)

2.50212143430033
2.3043426925835746
2.033662707210796
1.8216338777491488
1.6590379454747983
1.5263353050848971
1.4080705812165621
1.3101256252370712
1.2128665765525808


In [14]:
#Test for different alpha and beta parameters
cs = []
avg_cs = []
c1s = []
for a in [1,2,3,4,5,6,7,8,9,10]:
    for b in [1,2,3,4,5,6,7,8,9,10]: #test for a variety of alphas and betas
        for size in [200,500,1000,5000,10000,int(1e5)]:
            for seed in [1356,1023,1748,489,1265,31,1776,888,1901,1437,619,1975,509,1342,745,1623,1288,1755,94,1389]:
                rng = np.random.default_rng(seed)
                sample = rng.beta(a,b,size)
                cutoff = np.percentile(sample, 66.7) 
                test_sample = sample[sample > cutoff]
                sd = np.std(test_sample)
                c = (1-np.max(test_sample)) / sd
                cs.append(c)
            avg_c = np.average(cs)
            avg_cs.append(avg_c)
        c1 = np.average(avg_cs)
        c1s.append((a,b,c1))        

In [15]:
import plotly.graph_objects as go

x, y, z = zip(*c1s)
z = list(map(float, z))
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', 
                                   marker=dict(
                                   size=12,
                                   color=z,                # set color to an array/list of desired values
                                   colorscale='Viridis',   # choose a colorscale
                                   opacity=0.8
    ))])
fig.update_layout(scene = dict(
                    xaxis_title='alpha',
                    yaxis_title='beta',
                    zaxis_title='scaling constant'),
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

In [16]:
#Adjust sample cutoff by skew
cs = []
avg_cs = []
c2s = []
for a in [1,2,3,4,5,6,7,8,9,10]:
    for b in [1,2,3,4,5,6,7,8,9,10]: #test for a variety of alphas and betas
        for size in [200,500,1000,5000,10000,int(1e5)]:
            for seed in [1356,1023,1748,489,1265,31,1776,888,1901,1437,619,1975,509,1342,745,1623,1288,1755,94,1389]:
                rng = np.random.default_rng(seed)
                sample = rng.beta(a,b,size)
                cutoff = np.percentile(sample, 66.7*(1.01)**sc.skew(sample))
                test_sample = sample[sample > cutoff]
                sd = np.std(test_sample)
                c = (1-np.max(test_sample)) / sd
                cs.append(c)
            avg_c = np.average(cs)
            avg_cs.append(avg_c)
        c2 = np.average(avg_cs)
        c2s.append((a,b,c2)) 

In [17]:
x, y, z = zip(*c2s)
z = list(map(float, z))
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', 
                                   marker=dict(
                                   size=12,
                                   color=z,                # set color to an array/list of desired values
                                   colorscale='Viridis',   # choose a colorscale
                                   opacity=0.8
    ))])
fig.update_layout(scene = dict(
                    xaxis_title='alpha',
                    yaxis_title='beta',
                    zaxis_title='scaling constant'),
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()
fig2 = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, color='rgba(244,22,100,0.6)')])
fig2.update_layout(scene = dict(
                    xaxis_title='alpha',
                    yaxis_title='beta',
                    zaxis_title='scaling constant'),
                    margin=dict(r=20, b=10, l=10, t=10))
fig2.show()